In [20]:
# 1: This cell imports the necessary packages

import os
import pandas as pd
import numpy as np
import itertools

In [21]:
# Enter the 4 file paths and 1 directory (folder) path in the sections including: _file_path = "_insert_here_"

"""
Import 1: Campus Properties CSV
> This table contains enrolment information for each school campus, as well as properties that are used as inclusion/exclusion criteria.
> Each row represents data for a different campus (according to CampusId)
> The table MUST include the columns: "CampusId", "primary_enrolments", "secondary_enrolments"
"""
campus_properties_file_path = "_insert_here_"
campus_properties_data_types = {'CampusId' : str, 'School Type' : str,'primary_enrolments' : int, 'secondary_enrolments' : int }
campus_properties_columns = ["CampusId", "primary_enrolments", "secondary_enrolments"]


"""
Import 2: Building Properties CSV
> This table contains the BuildingId, condition & functionality for each building in the portfolio, as well as any properties that are used as inclusion/exclusion criteria.
> Each row represents data for a different building (according to BuildingId)
> The table MUST include the columns: "CampusId", "BuildingId", "Functionality", "Condition"
"""
building_properties_file_path = "_insert_here_"
building_properties_data_types = { 'CampusId' : str, 'BuildingId' : str, 'Condition' : float , 'Functionality' : float }
building_properties_columns = ["BuildingId", "CampusId", "Condition", "Functionality"]

"""
Import 3:  Inclusion Exclusion Register CSV
> This table specifies the criteria that are used to determine whether a given building or campus are included in the reducible space calculation.
> Each row represents a different exclusion criteria, which can be activated or deactivated using the column "exlcude_true_false"
> The table MUST include the columns:
"table_name": this value must be either 'Campus Properties' or 'Building Properties'. It is used to specify which table the exclusion is being applied to.
"column_name": this is the column within the specified table that the exlcusion is being applied to
"exclude_true_false": this indicates whether the exclusion is active or not; TRUE = active
"exclusion_type": this indicates whether the exclusion is applied using the 'num_range' columns, or the 'string_value_criteria' column. The two options for this column are 'num_range', or 'string_value'
"num_range_start": indicates where the exclusion range begins. I.e. down to and including the number that is specified
"num_range_stop": indicates where the exclusion range ends. I.e. up to and including the number that is specified
"string_value_criteria": indicates one or more string values which are then treated as exclusion criteria within the given column. Multiple values should be separated by comma's.
"""
inclusion_exclusion_file_path = "_insert_here_"
inclusion_exclusion_data_types = { 'filter_true_false' : bool, 'exclusion_type' : str, 'num_range_start' : float, 'num_range_stop' : float }
inclusion_exclusion_columns = ["table_name", "column_name",
                                "exclude_true_false", "exclusion_type",
                                "num_range_start", "num_range_stop", "string_value_criteria"]

"""
Import 4:  AIMS extract CSV
> This is an extract from AIMS that is commonly known as the 'building and room report'. It provides the data for calculating existing space at the FAS 1 - 2 level for each building.
> It can be found at this address: https://partner.eduweb.vic.gov.au/collaboration/AIMS/S1R1%20CORP/Forms/DET%20Document%20Set/docsethomepage.aspx?RootFolder=%2Fcollaboration%2FAIMS%2FS1R1%20CORP%2FS1%20R1%20AIMS%20REPORT%20EXAMPLES%2FRPT001%20-%20Extract&FolderCTID=0x0120007D803ED7D3E25A4587B6EED2CA7F9FB5&View=%7B8E2F392E-4991-44AF-9300-0D91D42638C7%7D
"""
AIMS_extract_file_path = "_insert_here_"
AIMS_report_data_types = { 'FloorArea' : float, 'CampusId' : str, 'BuildingId' : str, 'SpaceEntClass18' : str }
AIMS_report_columns = ["SpaceEntClass18", "CampusId", "FloorArea", "BuildingId"]

"""
Import 5:  FAS file folder
> This is a folder that contains 4 files which each provide different sections of the 2018 Facility Area Schedule. They are:
1. "FAS 1 Categories"
2. "FAS 2 Categories"
3. "Primary FAS"
4. "Secondary FAS"

Note: These MUST retain the same structure in order for the model to work, although if required, additional enrolment bands can be added without causing errors. The names for each must also stay consistent.
"""

FAS_folder_path = "_insert_here_"

In [22]:
FAS_folder_path = r"C:\Users\justi\Desktop\Reducible Space Folder\Test Files\FAS Files"
AIMS_extract_file_path = r"C:\Users\justi\Desktop\Reducible Space Folder\Test Files\ASSETS_20230310.csv"
building_properties_file_path = r"C:\Users\justi\Desktop\Reducible Space Folder\Test Files\Building Properties - Example Schools.csv"
campus_properties_file_path = r"C:\Users\justi\Desktop\Reducible Space Folder\Test Files\Campus Properties.csv"
inclusion_exclusion_file_path = r"C:\Users\justi\Desktop\Reducible Space Folder\Test Files\Inclusion-Exclusion Register.csv"

inclusion_exclusion_data_types = { 'filter_true_false' : bool, 'exclusion_type' : str, 'num_range_start' : float, 'num_range_stop' : float }
inclusion_exclusion_columns = ["table_name", "column_name",
                                "exclude_true_false", "exclusion_type",
                                "num_range_start", "num_range_stop", "string_value_criteria"]

AIMS_report_data_types = { 'FloorArea' : float, 'CampusId' : str, 'BuildingId' : str, 'SpaceEntClass18' : str }
AIMS_report_columns = ["SpaceEntClass18", "CampusId", "FloorArea", "BuildingId"]

building_properties_data_types = { 'CampusId' : str, 'BuildingId' : str, 'Condition' : float , 'Functionality' : float }
building_properties_columns = ["BuildingId", "CampusId", "Condition", "Functionality"]

campus_properties_data_types = {'CampusId' : str, 'School Type' : str,'primary_enrolments' : int, 'secondary_enrolments' : int }
campus_properties_columns = ["CampusId", "primary_enrolments", "secondary_enrolments"]


In [23]:
# 3: These functions are used to check whether the file contains the right columns and data types

def check_csv_columns(df, expected_columns):
    columns = df.columns.tolist()
    
    naughty_list = []
    
    for i in expected_columns:
        if i in columns:
            continue
        else:
            naughty_list.append(i)
    
    if len(naughty_list) > 0:
        raise ValueError(f"The following columns were expected but not found: {naughty_list}. Try checking for spelling/grammar differences or add the required columns.")

def convert_columns(df, column_types):
    for column, dtype in column_types.items():
        if column in df.columns:
            df[column] = df[column].astype(dtype)
        else:
            df[column] = pd.Series(dtype=dtype)

In [24]:
# This function iterates through the file names specified in the file_names list, attaching them to the folder path and retrieving the CSV file with the given name
# It then processes the CSV files into the format that is used in the model, and assigns them to a dictionary

def upload_fas(fas_folder_path):
    
    file_names = ['FAS 1 Categories.csv', 'FAS 2 Categories.csv', 'Primary FAS.csv', 'Secondary FAS.csv']
    
    FAS_dataframes = { }

    for file in os.listdir(fas_folder_path):
        for idx, target_file in enumerate(file_names):
            if file == target_file:
                file_path = os.path.join(fas_folder_path, file)
                df = pd.read_csv(file_path)
                FAS_dataframes[target_file] = df                 
                break

    """Below is the method for processing the FAS1-2 Category dataframes"""
        
    FAS_1_categories = FAS_dataframes['FAS 1 Categories.csv']
    FAS_2_categories = FAS_dataframes['FAS 2 Categories.csv']
    FAS_1_categories = FAS_1_categories.set_index("Facility Area Schedule")
    FAS_2_categories = FAS_2_categories.set_index("Facility Area Schedule")

    FAS_1_categories = FAS_1_categories
    FAS_2_categories = FAS_2_categories


    """Below is the method for processing the Pri_FAS & Sec_FAS dataframes"""

    Pri_FAS = FAS_dataframes['Primary FAS.csv']
    Sec_FAS = FAS_dataframes['Secondary FAS.csv']

    # 2: rename columns to match later requirements
    Sec_FAS.rename(columns = {'Upper Enrolments': 'FAS Layer 2', 'FAS Level' : 'FAS Layer'}, inplace = True)
    Pri_FAS.rename(columns = {'Upper Enrolments': 'FAS Layer 2'}, inplace = True)

    # 3: Set the category names as the indices
    Sec_FAS_index = Sec_FAS['FAS Layer 2']
    Sec_FAS = Sec_FAS.set_index(Sec_FAS_index)
    Pri_FAS_index = Pri_FAS['FAS Layer 2']
    Pri_FAS = Pri_FAS.set_index(Pri_FAS_index)

    # 4: remove FAS categories column
    Sec_FAS = Sec_FAS.drop(['FAS Layer 2'], axis = 1)
    Pri_FAS = Pri_FAS.drop(['FAS Layer 2'], axis = 1)

    # 5: insert an enrolment band for '0' students
    Pri_FAS.insert(1, '0', 0)
    Sec_FAS.insert(1, '0', 0)

    # 6: insert a column classifying which enrolment type the row belongs to

    Pri_FAS.insert(1, 'Student Type', 'Primary')

    Sec_FAS.insert(1, 'Student Type', 'Secondary')

    # 7: Splits the indices into a multi-index depending on FAS Layer

    def split_index(FAS_df):
        FAS_1 = None
        new_index = []

        for n, index in enumerate(FAS_df.index):
            layer = FAS_df.iat[n, 0]

            if layer == 1:
                FAS_1 = index
            elif layer == 2:
                new_index.append((FAS_1, index))

        FAS_df = FAS_df[FAS_df['FAS Layer'] > 1]
        FAS_df.set_index(pd.MultiIndex.from_tuples(new_index, names=['FAS 1', 'FAS 2']), inplace=True)
        FAS_df = FAS_df.drop(columns=["FAS Layer"])

        return FAS_df

    Pri_FAS = split_index(Pri_FAS)

    Sec_FAS = split_index(Sec_FAS)

    # 8: Assigns the Core/Support label to each row (FAS Category)

    def core_support(FAS_df, FAS_2_categories):
        core_support_column = []
        FAS_1_values = FAS_df.index.get_level_values(0).unique()
        FAS_2_values = FAS_df.index.get_level_values(1)
        for FAS_2 in FAS_2_values:
            if FAS_2_categories.at[FAS_2, "Space classification"] == 'Core':
                core_support_column.append('Core')
            elif FAS_2_categories.at[FAS_2, "Space classification"] == 'Support':
                core_support_column.append('Support')   
            else:
                core_support_column.append('N/A')
        FAS_df.insert(0, "Space classification", core_support_column)
        return FAS_df

    Pri_FAS = core_support(Pri_FAS, FAS_2_categories)
    Sec_FAS = core_support(Sec_FAS, FAS_2_categories)

    FAS_dataframes_dict = { 'Pri_FAS' : Pri_FAS, 'Sec_FAS' : Sec_FAS, 'FAS_1_categories' : FAS_1_categories, 'FAS_2_categories' : FAS_2_categories }

    return FAS_dataframes_dict

FAS_dictionary = upload_fas(FAS_folder_path)

Pri_FAS = FAS_dictionary['Pri_FAS']
Sec_FAS = FAS_dictionary['Sec_FAS']
FAS_1_categories = FAS_dictionary['FAS_1_categories']
FAS_2_categories = FAS_dictionary['FAS_2_categories']

In [25]:
Pri_FAS.head()

Space classification  \
FAS 1         FAS 2                                                                     
Learning Base Shared Learning                                                    Core   
              Practical Activities                                               Core   
              Consult/Individual Learning Space                                  Core   
              Support Spaces (Kitchen, Laundry, Equipment Store)                 Core   
              Relocatable Buildings - Learning Base                              Core   

                                                                 Student Type  \
FAS 1         FAS 2                                                             
Learning Base Shared Learning                                         Primary   
              Practical Activities                                    Primary   
              Consult/Individual Learning Space                       Primary   
              Support Spaces (Kitchen, Laundry, Equipment Store)      Primary   
              Relocatable Buildings - Learning Base                   Primary   

                                                                  0  25  50  \
FAS 1         FAS 2                                                           
Learning Base Shared Learning                                     0  65  65   
              Practical Activities                                0   7   7   
              Consult/Individual Learning Space                   0   8   8   
              Support Spaces (Kitchen, Laundry, Equipment Store)  0   0   0   
              Relocatable Buildings - Learning Base               0   0  90   

                                                                   75  100  \
FAS 1         FAS 2                                                          
Learning Base Shared Learning                                     130  195   
              Practical Activities                                 14   21   
              Consult/Individual Learning Space                    16   24   
              Support Spaces (Kitchen, Laundry, Equipment Store)    0    0   
              Relocatable Buildings - Learning Base                90   90   

                                                                  125  150  \
FAS 1         FAS 2                                                          
Learning Base Shared Learning                                     260  325   
              Practical Activities                                 28   35   
              Consult/Individual Learning Space                    32   40   
              Support Spaces (Kitchen, Laundry, Equipment Store)    0    0   
              Relocatable Buildings - Learning Base                90   90   

                                                                  175  ...  \
FAS 1         FAS 2                                                    ...   
Learning Base Shared Learning                                     390  ...   
              Practical Activities                                 42  ...   
              Consult/Individual Learning Space                    48  ...   
              Support Spaces (Kitchen, Laundry, Equipment Store)    0  ...   
              Relocatable Buildings - Learning Base                90  ...   

                                                                  1275  1300  \
FAS 1         FAS 2                                                            
Learning Base Shared Learning                                     1040  1040   
              Practical Activities                                 112   112   
              Consult/Individual Learning Space                    128   128   
              Support Spaces (Kitchen, Laundry, Equipment Store)    10    10   
              Relocatable Buildings - Learning Base               3014  3096   

                                                                  1325  1350  \
FAS 1         FAS 2                  

In [26]:
def upload_csv(file_path, dtypes, columns):
    
    try:
        df = pd.read_csv(file_path)   #  <-- This converts the csv file into a pandas dataframe (table)
        check_csv_columns(df, columns)   #  <-- This is the function that tests whether all the required columns are included in the dataframe
    except:
        print(f"Failed to read file: {file_path}")
        
    convert_columns(df, dtypes)
    
    return df

In [27]:
inclusion_exclusion_df = upload_csv(inclusion_exclusion_file_path, inclusion_exclusion_data_types, inclusion_exclusion_columns)

In [28]:
campus_properties_df = upload_csv(campus_properties_file_path, campus_properties_data_types, campus_properties_columns)

In [29]:
building_properties_df = upload_csv(building_properties_file_path, building_properties_data_types, building_properties_columns)

In [30]:
def upload_aims_report(file_path, dtypes, columns):
    
    aims_report_df = pd.read_csv(file_path)
    
    # This converts the columns to the right data type
    convert_columns(aims_report_df, dtypes)

    check_csv_columns(aims_report_df, columns)
        
    actual_space_extract = aims_report_df.loc[:,['CampusId', 'SchoolNo', 'CampusName', 'BuildingId', 'BuildingName', 'SpaceId','SpaceEntClass18', 'FloorArea', 'Classification']]

    # This produces a dataframe with all the unique SpaceEntClass18 values broken into the constituent values e.g. so that they can be matched to FAS 1 & 2 categories
    space_classifications = aims_report_df['SpaceEntClass18'].unique()
    classification_index = pd.DataFrame(space_classifications, columns=['space_classifications']).set_index('space_classifications')
    classification_index[['year', 'standard', 'FAS 1', 'FAS 2', 'FAS 3']] = classification_index.index.to_series().str.split('\\\\', expand=True)     

    

    # filtering out any rows with null / not numerical values for floor area
    actual_space_extract = actual_space_extract.loc[(actual_space_extract["FloorArea"] > -1)]

    # this assigns two new columns to the actual_space_extract file
    actual_space_extract = actual_space_extract.assign(**{"FAS 1": "", "FAS 2": ""})

    # Uses 'map' to retrieve the FAS 1 & 2 categories from the classification index according to the SpaceEntClass18 value in the AIMS extract
    actual_space_extract['FAS 1'] = actual_space_extract['SpaceEntClass18'].map(classification_index['FAS 1'])
    actual_space_extract['FAS 2'] = actual_space_extract['SpaceEntClass18'].map(classification_index['FAS 2'])

    # Uses `str.strip()` to remove leading and trailing spaces from the columns
    actual_space_extract['FAS 1'] = actual_space_extract['FAS 1'].str.strip()
    actual_space_extract['FAS 2'] = actual_space_extract['FAS 2'].str.strip()

    aims_report_df = actual_space_extract

    core_space_categories = Pri_FAS[Pri_FAS['Space classification'] == 'Core'].index.get_level_values(1).tolist() + Sec_FAS[Sec_FAS['Space classification'] == 'Core'].index.get_level_values(1).tolist()

    FAS_2_space = aims_report_df.pivot_table(index = 'BuildingId', columns = 'FAS 2',
                                                      values='FloorArea',  aggfunc = 'sum',  fill_value = 0)

    FAS_1_space = aims_report_df.pivot_table(index = 'BuildingId', columns = 'FAS 1',
                                                      values='FloorArea',  aggfunc = 'sum',  fill_value = 0)

    FAS_2_core_space = FAS_2_space.filter(items = core_space_categories)
    
    return aims_report_df, core_space_categories, FAS_2_space, FAS_1_space, FAS_2_core_space

aims_report_outputs = upload_aims_report(AIMS_extract_file_path, AIMS_report_data_types, AIMS_report_columns)

C:\Users\justi\AppData\Local\Temp\ipykernel_22380\770144173.py:3: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  aims_report_df = pd.read_csv(file_path)


In [31]:


aims_report_df = aims_report_outputs[0]
aims_report_df.head(3)

,CampusId,SchoolNo,CampusName,BuildingId,BuildingName,SpaceId,SpaceEntClass18,FloorArea,Classification,FAS 1,FAS 2
0,1518901,5189,Cranbourne West Primary School,426975,Relocatable 012547,1299791,2018 \ Primary-STD \ Covered Outdoor and Outdo...,1.5,BUILDING,Covered Outdoor and Outdoor Travel,Outdoor Travel - Relocatable
1,1518901,5189,Cranbourne West Primary School,426975,Relocatable 012547,1299792,2018 \ Primary-STD \ Staff Work \ Staff Work \...,11.9,BUILDING,Staff Work,Staff Work
2,1518901,5189,Cranbourne West Primary School,426975,Relocatable 012547,1299793,2018 \ Primary-STD \ Covered Outdoor and Outdo...,1.5,BUILDING,Covered Outdoor and Outdoor Travel,Outdoor Travel - Relocatable


In [32]:
# This is a list of the FAS 2 categories that are considered çore

core_space_categories = aims_report_outputs[1]
core_space_categories[0:3]

['Shared Learning',
 'Practical Activities',
 'Consult/Individual Learning Space']

In [33]:
# This is a dataframe where each row is for a given building (index = BuildingId), and the columns indicate the amount of aggregated floor area within each FAS 2 space type

FAS_2_space = aims_report_outputs[2]
FAS_2_space.loc['101274']

FAS 2
Biology,General Science                               0.0
Bus Shelter                                           0.0
Canteen                                               0.0
Ceramics,Sculpture                                    0.0
Chemistry                                             0.0
                                                     ... 
Support Spaces (Kitchen, Laundry, Equipment Store)    0.0
Support Spaces,Meeting Rooms                          0.0
Textiles                                              0.0
Visual Arts                                           0.0
Visual Communication                                  0.0
Name: 101274, Length: 66, dtype: float64

In [34]:
# This is a dataframe where each row is for a given building (index = BuildingId), and the columns indicate the amount of aggregated floor area within each FAS 1 space type

FAS_1_space = aims_report_outputs[3]

print(FAS_1_space.loc['105587'])
print(FAS_1_space.loc['105585'])

FAS_1_space

FAS 1
Administration                                          8.0
Circulation                                           208.0
Cleaning, Maintenance and Storage                     165.0
Covered Outdoor and Outdoor Travel                     51.0
Learning Base                                          29.0
Learning Resource Centre                                0.0
No Entitlement                                        879.0
Performing Arts, PE, Canteen, Food Technology          74.0
Resource Intensive Learning                            25.4
School Community Hub                                    0.0
Science                                                 0.0
Staff Work                                             31.0
Student and Staff Amenities                            82.0
Student and Staff Amenities - SDS schools only          0.0
Student and Staff Amenities - Special schools only      0.0
Technology                                             79.0
The Arts - Fine Arts and Multimedi

FAS 1,Administration,Circulation,"Cleaning, Maintenance and Storage",Covered Outdoor and Outdoor Travel,Learning Base,Learning Resource Centre,No Entitlement,"Performing Arts, PE, Canteen, Food Technology",Resource Intensive Learning,School Community Hub,Science,Staff Work,Student and Staff Amenities,Student and Staff Amenities - SDS schools only,Student and Staff Amenities - Special schools only,Technology,The Arts - Fine Arts and Multimedia
BuildingId,,,,,,,,,,,,,,,,,
101274,0.0,0.00,0.0,0.0,0.0,0.0,1124.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101275,0.0,0.00,0.0,58.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101276,382.0,955.00,17.0,554.0,1093.0,35.0,0.0,0.0,221.0,174.00,0.0,52.0,167.0,0.0,0.0,0.0,0.0
101286,107.1,48.20,0.0,0.0,222.0,198.0,0.0,0.0,0.0,0.00,0.0,17.0,13.0,0.0,0.0,0.0,0.0
101287,69.0,152.67,0.0,217.0,0.0,168.0,8.0,0.0,102.0,329.54,0.0,46.0,109.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846630,0.0,0.00,12.3,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
846636,0.0,289.40,16.4,223.7,198.6,25.9,300.4,719.0,0.0,0.00,0.0,21.4,35.8,0.0,0.0,0.0,584.3
846639,37.2,557.00,8.5,228.0,1292.5,248.2,77.7,0.0,0.0,0.00,0.0,83.2,35.9,0.0,0.0,0.0,77.0


In [35]:
# This is a dataframe where each row is for a given building (index = BuildingId), and the columns indicate the amount of aggregated core space within each FAS 2 space type

# I.e. same as FAS_2_space dataframe, except filtered to exlcude non-core space categories from the columns

FAS_2_core_space = aims_report_outputs[4]

FAS_2_core_space.head(3)

FAS 2,Shared Learning,Practical Activities,"Support Spaces (Kitchen, Laundry, Equipment Store)",Relocatable Buildings - Learning Base,Practical Activities - Art,Practical Activities - Food Preparation,Relocatable Buildings - Resource Intensive Learning,Multi-Purpose Space,Physical Education Gymnasium,Music,...,Relocatable Buildings - Canteen,Library,Senior Personnel Work Area,Staff Centre,First Aid,Relocatable Buildings - Staff Admin,Staff Work,"Staff Amenities (Toilets, Shower, Change)","Student Amenities (Toilets, Shower, Change)",Relocatable Buildings - Toilets and Student Centre
BuildingId,,,,,,,,,,,,,,,,,,,,,
101274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101276,1093.0,0.0,0.0,0.0,221.0,0.0,0.0,50.0,0.0,46.0,...,0.0,35.0,111.0,183.0,23.0,0.0,52.0,23.0,144.0,0.0


In [36]:
import random

p_core = Pri_FAS[Pri_FAS.index.get_level_values(1).isin(core_space_categories)]
s_core = Sec_FAS[Sec_FAS.index.get_level_values(1).isin(core_space_categories)]

pri_entitlement = p_core.groupby(level='FAS 1')['100'].sum()
sec_entitlement = s_core.groupby(level = 'FAS 1')['75'].sum()

test_entitlement = pri_entitlement.add(sec_entitlement, fill_value = 0)

total_entitled = test_entitlement.sum()
print(total_entitled)

test_core_space = test_entitlement.apply(lambda x: x*random.uniform(0.5, 5) + 500)

test_core_space = test_core_space.apply(round)

test_excess_space =  test_core_space.sub(test_entitlement, fill_value = 0)

test_core_space

1748.0


FAS 1
Administration                                    750
Learning Base                                    2212
Learning Resource Centre                          615
Performing Arts, PE, Canteen, Food Technology     579
Resource Intensive Learning                       540
School Community Hub                              500
Science                                          1032
Staff Work                                        579
Student and Staff Amenities                      1081
Technology                                        764
The Arts - Fine Arts and Multimedia              1070
dtype: int64

In [40]:
combo_tuple_test = ('105587', '105585')

building_1 = Building('105587', building_properties_df, FAS_2_space, FAS_2_core_space, aims_report_df)

building_2 = Building('105585', building_properties_df, FAS_2_space, FAS_2_core_space, aims_report_df)

building_dictionary = {'105587' : building_1, '105585' : building_2}



class Combination():
    def __init__(self, combination_tuple, FAS_1_excess_core_space, building_dictionary, FAS_1_core_entitlement, FAS_1_core_space, entitled_core_space):

        FAS_1_excess_core_space = FAS_1_excess_core_space.apply(lambda x: x if x >= 0 else 0) # <-- this applies a function which converts negative values to zero. Output: pd.Series

        combination_space = FAS_1_core_entitlement

        combination_space = combination_space.apply(lambda x: 0) # <-- creates a fresh pd.Series with all the FAS 1 categories and 0 for actual space

        # This loop finds the amount of space in each FAS 1 category
        for ID in combination_tuple:

            # retrieves the building object from the building dictionary created for the current campus object
            building = building_dictionary[ID]
            
            building_space = building.FAS_1_core_space["Actual Space"]

            # adds the building space to the pd.series representing the combination's space
            combination_space = combination_space.add(building_space, fill_value = 0)
            
        # This is the "actual space" for the combination of buildings
        self.combination_FAS_1_core_space = combination_space

        self.FAS_1_remaining_excess_space = FAS_1_excess_core_space.sub(combination_space, fill_value = 0) # <--  subtracts the combination area from the excess core space to find the remaining excess space at each FAS 1 level
        
        self.FAS_1_remaining_core_space = FAS_1_core_space.sub(combination_space, fill_value = 0) #  <-- This subtracts the combination area from the core space to find the remaining space at each FAS 1 level

        self.remaining_total_space = self.FAS_1_remaining_core_space.sum() #  <-- This is a float value that shows the sum of core space
        
        
        # This finds the percentage of excess space remaining after the combination is removed
        actual = self.remaining_total_space
        entitled = entitled_core_space
        self.percentage_excess_core_space = ((actual - entitled) / actual ) * 100
        
        """ This section classifies the combination as reducible or not reducible"""
        
        reducible_classification = True
        
        for i in self.FAS_1_remaining_excess_space:
            if i < 0:
                reducible_classification = False
        
        if self.percentage_excess_core_space < 0:
            reducible_classification = False
            
        self.reducible_classification = reducible_classification

        self.reducible_space = self.combination_FAS_1_core_space.sum()

test_combo = Combination(combo_tuple_test, test_excess_space, building_dictionary, test_entitlement, test_core_space, total_entitled)

print(type(test_combo.reducible_space))

test_combo.reducible_space

<class 'numpy.float64'>


1370.8

In [41]:
# This is the building class. It is used to aggregate space at the campus level and find the space for different combinations of buildings

class Campus():
    def __init__(self, campus_id, campus_properties, aims_extract, FAS_2_space, FAS_2_core_space, building_properties, inclusion_exclusion_df):
        
        self.enrolment_categories_tuple = self.enrolmentment_categories(campus_id, campus_properties, FAS_1_categories, FAS_2_categories, Sec_FAS, Pri_FAS) # # # !!! The FAS category data will need to have the PageOne class specified        
        
        buildings = building_properties[building_properties['CampusId'] == campus_id]
        
        self.buildings = buildings['BuildingId'].unique()
        
        self.Pri_entitled = self.FAS_1_2_entitlement_calc(Pri_FAS, Sec_FAS, self.enrolment_categories_tuple)[0] # !!! The FAS category data will need to have the PageOne class specified
        
        self.Sec_entitled = self.FAS_1_2_entitlement_calc(Pri_FAS, Sec_FAS, self.enrolment_categories_tuple)[1] # !!! The FAS category data will need to have the PageOne class specified
        
        self.FAS_2_entitlement = self.FAS_2_entitlement_calc(self.Pri_entitled, self.Sec_entitled, FAS_2_categories)[0]
        
        self.FAS_2_core_entitlement = self.FAS_2_entitlement_calc(self.Pri_entitled, self.Sec_entitled, FAS_2_categories)[1]
        
        self.FAS_1_core_entitlement = self.FAS_1_core_entitlement # <-- This is a pd.Series that shows the entitlement at the FAS 1 level
        
        self.FAS_1_entitlement = self.FAS_1_entitlement # <-- Same as above, except with non-core FS 2 categories filtered out
        
        self.entitled_core_space = self.FAS_1_core_entitlement.sum() # <-- This finds the sum of entitled core space i.e  ENTITLED CORE SPACE FOR THE CAMPUS
        
        self.buildings_dictionary = {}
        
        for ID in self.buildings:
            self.buildings_dictionary[ID] = Building(ID, building_properties, FAS_2_space, FAS_2_core_space, aims_extract)
            
            
        """
        Building attributes:
        1. self.FAS_2_space
        2. self.FAS_1_space
        3. self.FAS_1_core_space
        4. self.FAS_1_2_core_space
        
        The below section calculates the total & core space for the campus:
        """
        
        FAS_1_space_table_list = []
        FAS_1_core_space_table_list = []
        
        for i in self.buildings:
            FAS_1_space_table = self.buildings_dictionary[i].FAS_1_space
            FAS_1_space_table_list.append(FAS_1_space_table)
            
            FAS_1_core_space_table = self.buildings_dictionary[i].FAS_1_core_space
            FAS_1_core_space_table_list.append(FAS_1_core_space_table)
            
        self.FAS1_merged = pd.concat(FAS_1_space_table_list, axis = 0)
        self.FAS_1_space = self.FAS1_merged.sum()
        self.total_space = self.FAS_1_space.sum()

        self.FAS1_core_merged = pd.concat(FAS_1_core_space_table_list, axis = 1, join = 'outer')
        self.FAS1_core_merged.fillna(0, inplace=True)
        
        # This is a pd.Series that shows the core space for each FAS 1 category
        self.FAS_1_core_space = self.FAS1_core_merged.sum(axis = 1)
        
        # This is a float value that shows the sum of core space
        self.total_core_space = self.FAS_1_core_space.sum()  #  <-- TOTAL CORE SPACE FOR CAMPUS
        
        """
        The below section calculates the excess core space for the campus:
        """
        
        actual = self.total_core_space
        entitled = self.entitled_core_space

        self.percentage_excess_core_space = ((actual - entitled) / actual ) * 100

        # This provides the excess floor area at each FAS 1 level
        self.FAS_1_excess_core_space = self.FAS_1_core_space.sub(self.FAS_1_core_entitlement, fill_value = 0)
        
        """
        The below section calculates the combinations of reducible buildings:
        """
        
        building_exclusions = inclusion_exclusion_df.loc[(inclusion_exclusion_df["table_name"] == "Building Properties") & (inclusion_exclusion_df["exclude_true_false"] == True)]
        
        numeric_exclusions = building_exclusions.loc[inclusion_exclusion_df["exclusion_type"] == 'num_range']
        
        string_exclusions = building_exclusions.loc[inclusion_exclusion_df["exclusion_type"] == 'string_value']
        
        exclusion_list = [] # <-- This is the list where excluded buildings are stored
        
        # iterates through the columns in the numeric exclusions included in the inclusion_exclusion dataframe
        for exclusion in numeric_exclusions["column_name"]:
            
            # checks if the exclusion is included in the building_properties dataframe
            if exclusion in list(buildings.columns):
                
                # defines the lower & upper thresholds for the exclusion criteria ( .iloc[0] is used to retrieve the first value from the pd.Series that is returned by .loc )
                low_value = numeric_exclusions.loc[numeric_exclusions["column_name"] == exclusion, "num_range_start"].iloc[0]
                high_value = numeric_exclusions.loc[numeric_exclusions["column_name"] == exclusion, "num_range_stop"].iloc[0]
                
                # Iterates through the buildings in the building_properties dataframe; actual_value is the number for the given building 
                for building in buildings["BuildingId"]:
                    actual_value = buildings.loc[buildings["BuildingId"] == building, exclusion].iloc[0]
                    
                    # checks if value is within exclusion range
                    try:
                        if actual_value >= low_value and actual_value <= high_value:
                            exclusion_list.append(building)
                    except:
                        print(f"Actual value: {type(actual_value)}, high-low: {high_value},{low_value}")
                        
        # iterates through the columns in the string_value exclusions included in the inclusion_exclusion dataframe
        for exclusion in string_exclusions["column_name"]:
            
            # checks if the exclusion is included in the building_properties dataframe
            if exclusion in list(buildings.columns):
                
                # This retrieve the comma-separated strings
                comma_separated_strings = string_exclusions.loc[string_exclusions["column_name"] == exclusion, "string_value_criteria"].iloc[0]
                
                # This breaks up the string into the individual and strips any spaces from the start/finish
                string_list = comma_separated_strings.split(",")
                string_list = [s.strip() for s in string_list]
                
                for building in buildings["BuildingId"]:
                    actual_value = buildings.loc[buildings["BuildingId"] == building, exclusion].iloc[0]
                    
                    # checks if the building attribute (actual_value) is in the list of exclusions
                    try:
                        if actual_value in string_list:
                            exclusion_list.append(building)
                    except:
                        print(f"Building: {building}, Actual value: {actual_value}, Exclusion list: {string_list}") # <-- This prints the attributes that caused an error to occur
        
        
        # This defines the list of buildings that are considered 'reducible'
        included_buildings = [b for b in buildings["BuildingId"] if b not in exclusion_list]
        
        """
        This section finds the combinations of reducible buildings
        """        

        """
        This converts negative values in the excess space FAS 1 table to zero:
        > This is so that we can detect whether the reduction of this combination would affect FAS 1 categories which are already under-entitled
        """ 
        
        # This finds all the combinations of reducible schools
        combinations_list = []
        for i in range(len(included_buildings)):
            combo = list(itertools.combinations(included_buildings, i+1))
            combinations_list.append(combo)
              
        # This converts the list into a continuous list of tuples, rather than a list of lists for each iteration
        combinations_list = [item for sublist in combinations_list for item in sublist]
 
        # this defines all the combinations within a dictionary using the format (combination tuple) : combination_object
        combinations_dict = {}
        for combo_tuple in combinations_list:
            combinations_dict[combo_tuple] = Combination(combo_tuple, self.FAS_1_excess_core_space, self.buildings_dictionary, self.FAS_1_core_entitlement, self.FAS_1_core_space, self.entitled_core_space)      
        
        self.combinations = combinations_dict   
        
        combination_rank = self.find_highest_fas1_core_space(combinations_dict)
        
        self.reducible_space = combination_rank[1]
        
        self.reducible_combination = combination_rank[0]
        
        
        
    def find_highest_fas1_core_space(self, dictionary):
    
        highest_fas1_key = None
        highest_fas1_value = float('-inf') # <-- Sets the core space to negative infitinity to make sure that the first FAS 1 core space value is accepted

        for key, combination in dictionary.items():
            #print(key, combination)
            #print(combination.reducible_space)
            if combination.reducible_classification == True: # <-- tests whether the combination is classed as reducible or not
                if combination.reducible_space > highest_fas1_value:
                    #print(key, combination)
                    highest_fas1_key = key
                    highest_fas1_value = combination.reducible_space

        return highest_fas1_key, highest_fas1_value
    

    def enrolment_category(self,enrolment, max_val):
        
        # Generate a range of values at intervals of 25 using numpy
        bins = np.arange(0, max_val + 25, 25)
        if enrolment in bins:
            category = enrolment
            # Use numpy digitize to categorize the enrollment
        else:
            category_idx = np.digitize(enrolment, bins)
            # Return the highest value in the category
            category = bins[category_idx] if category_idx > 0 else None

        return category
    
    def enrolment_category1(self, enrolment, max_val):
        # Generate a range of values at intervals of 25 using numpy
        bins = np.arange(0, max_val + 25, 25)
        if enrolment in bins:
            category = enrolment
        else:
            category_idx = np.digitize(enrolment, bins)
            if category_idx < len(bins):
                category = bins[category_idx]
            else:
                category = None
        return category
    
    
    def enrolmentment_categories(self, campus_ID, campus_properties, FAS_1_categories, FAS_2_categories, Sec_FAS, Pri_FAS):
        
        # this gets the school based on the campus input, which is used to index the school_data dataframe
        school = campus_properties.loc[campus_properties['CampusId'] == campus_ID]
        
        # Converts the campus_id values in the campus properties to string values (should already be done but just a check)        
        
        primary_FAS_max_enrolments = (len(list(Pri_FAS.columns)) - 2) * 25 # this finds the number of the FAS categories (i.e. maximum number of primary students in the FAS)

        secondary_FAS_max_enrolments = (len(list(Sec_FAS.columns)) - 2) * 25 # this finds the number of the FAS categories (i.e. maximum number of secondary students in the FAS)

        primary_enrolments = campus_properties.loc[campus_properties['CampusId'] == campus_ID, "primary_enrolments"].item() # this retrieves the number of primary enrolments from the school_data for the given campus
        
        secondary_enrolments = campus_properties.loc[campus_properties['CampusId'] == campus_ID, "secondary_enrolments"].item() # this retrieves the number of secondary enrolments from the school_data for the given campus

        primary_enrolment_category = self.enrolment_category(primary_enrolments, primary_FAS_max_enrolments) # this finds the enrolment band for the primary FAS

        secondary_enrolment_category = self.enrolment_category(secondary_enrolments, secondary_FAS_max_enrolments) # this finds the enrolment band for the secondary FAS

        enrolment_categories_tuple = (str(primary_enrolment_category), str(secondary_enrolment_category))
        
        return enrolment_categories_tuple
    
    # This returns the entitled space for both primary and secondary FAS 2 categories as two dataframes in a tuple)
    def FAS_1_2_entitlement_calc(self, Pri_FAS, Sec_FAS, enrolment_categories):
        
        primary_enrolments_category = enrolment_categories[0]
        
        # Removes the enrolment bands from the Pri_FAS df and keeps the core/support classification
        Pri_entitled = Pri_FAS['Space classification']
        
        # Converts the pd.Series into a dataframe
        Pri_entitled = Pri_entitled.to_frame(name='Space classification')
        
        FAS_2_index = list(Pri_entitled.index)
        
        primary_entitlement_list = []
    
        # This loop finds the entitled space for each FAS 2 category, and has some (hopefully redundent) code for handling index duplicates and weird stuff
        for i in FAS_2_index:
            entitlement = Pri_FAS.loc[i, primary_enrolments_category]
            if type(entitlement) == pd.Series:
                entitlement = entitlement.mean()
            primary_entitlement_list.append(entitlement)
            
        Pri_entitled["Entitled Space"] = primary_entitlement_list
        
        secondary_enrolments_category = enrolment_categories[1]
        
        # Removes the enrolment bands from the Pri_FAS df and keeps the core/support classification
        Sec_entitled = Sec_FAS['Space classification']
        
        # Converts the pd.Series into a dataframe
        Sec_entitled = Sec_entitled.to_frame(name='Space classification')
        
        FAS_2_index = list(Sec_entitled.index)
        
        sec_entitlement_list = []
        
        for i in FAS_2_index:
            entitlement = Sec_FAS.loc[i, secondary_enrolments_category]
            if type(entitlement) == pd.Series:
                entitlement = entitlement.mean()
            sec_entitlement_list.append(entitlement)
            
        Sec_entitled["Entitled Space"] = sec_entitlement_list

        return Pri_entitled, Sec_entitled
    
    def FAS_2_entitlement_calc(self, pri_entitled, sec_entitled, FAS_2_categories):
        
        # This retrieves the FAS 2 categories from the FAS_2 categories dataframe
        FAS_2_entitlement = FAS_2_categories
        
        FAS_2_core_entitlement = FAS_2_categories.loc[FAS_2_categories["Space classification"] == "Core"]    
        
        idx = pd.IndexSlice # <--- This is used to deal with the multi-index in the pre/sec entitled dataframes
            
        entitlement_list = []
        
        for i in FAS_2_entitlement.index:
            entitled_space = 0
            if i in pri_entitled.index.get_level_values(1):
                entitlement = pri_entitled.loc[idx[:,i], "Entitled Space"].sum()
                entitled_space += entitlement
                
            if i in sec_entitled.index.get_level_values(1):
                entitlement = sec_entitled.loc[idx[:,i], "Entitled Space"].sum()
                entitled_space += entitlement
            #print(f"Entitlement: {i}; Area:", entitled_space) # <-- This can be used to check the result
            entitlement_list.append(entitled_space)
            
        FAS_2_entitlement["Entitled Space"] = entitlement_list
        
        core_entitlement_list = []
        
        for i in FAS_2_core_entitlement.index:
            entitled_space = 0
            if i in pri_entitled.index.get_level_values(1):
                entitlement = pri_entitled.loc[idx[:,i],"Entitled Space"].sum()
                entitled_space += entitlement
            if i in sec_entitled.index.get_level_values(1):
                entitlement = sec_entitled.loc[idx[:,i],"Entitled Space"].sum()
                entitled_space += entitlement
            #print(f"Entitlement: {i}; Area:", entitled_space)  # <-- This can be used to check the result
            core_entitlement_list.append(entitled_space)
        
        FAS_2_core_entitlement["Entitled Space"] = core_entitlement_list
        FAS_2_core_entitlement.loc[:, "Entitled Space"] = core_entitlement_list
        
        
        # The below section uses the FAS 2 entitlements to aggregate at the FAS 1 level
        
        self.primary_FAS_1_entitlement = pri_entitled.groupby(level=0)['Entitled Space'].sum()
        self.secondary_FAS_1_entitlement = sec_entitled.groupby(level=0)['Entitled Space'].sum()
        
        self.FAS_1_entitlement = self.primary_FAS_1_entitlement.add(self.secondary_FAS_1_entitlement, fill_value = 0)
        
        core_pri_entitled = pri_entitled.loc[pri_entitled["Space classification"] == "Core"]
        core_sec_entitled = sec_entitled.loc[sec_entitled["Space classification"] == "Core"]
        
        self.primary_FAS_1_core_entitlement = core_pri_entitled.groupby(level=0)['Entitled Space'].sum()
        self.secondary_FAS_1_core_entitlement = core_sec_entitled.groupby(level=0)['Entitled Space'].sum()
        
        self.FAS_1_core_entitlement = self.primary_FAS_1_core_entitlement.add(self.secondary_FAS_1_core_entitlement, fill_value = 0)
        
        return FAS_2_entitlement, FAS_2_core_entitlement

class Building(Campus):
    def __init__(self, building_id, building_properties, FAS_2_space, FAS_2_core_space, aims_extract):
        
        """
        Attributes are:
        
        1. self.FAS_2_space
        2. self.FAS_1_space
        3. self.FAS_1_core_space
        4. self.FAS_1_2_core_space
        
        """
        
        self.building_id = building_id
        self.exclusion_attributes = building_properties[building_properties['BuildingId'] == building_id]
        self.FAS_2_space = FAS_2_space.loc[[building_id]]        
        self.FAS_1_space = FAS_1_space.loc[[building_id]]
        
        
        # This returns an abbreviated version of the AIMS BRR report which only contains spaces which match the building Id
        self.aims_extract = aims_extract[aims_extract['BuildingId'] == building_id].filter(items = ['BuildingId', 'FAS 1', 'FAS 2', 'FloorArea'])
        
        
        # This finds the FAS 2 categories present in the current building
        FAS_2_building_categories = self.aims_extract['FAS 2'].unique()
        
        self.FAS_1_2_data = pd.DataFrame(index = FAS_2_building_categories, columns = ['FAS 1', 'Space Classification', 'Actual Space'])
        
        # This loop finds the FAS 1 category, floor area and Core/Support category for each FAS 2 category in the building
        for FAS_2 in FAS_2_building_categories:
            try:
                matching_FAS_1 = self.aims_extract.loc[self.aims_extract['FAS 2'] == FAS_2, 'FAS 1'].tolist()
                floor_area = self.FAS_2_space.at[building_id, FAS_2]
            # This handles buildings which randomly throw up errors (quite rare but seems to happen)
            except:
                continue
            try:
                matching_category = FAS_2_categories.at[FAS_2, 'Space classification'] # # # !!!!! FAS_2_Categories needs to changed to call on thr PageOne class
            except KeyError:
                matching_category = 'Support'
            
            self.FAS_1_2_data.at[FAS_2,'FAS 1'] = matching_FAS_1[0]
            self.FAS_1_2_data.at[FAS_2,'Actual Space'] = floor_area
            self.FAS_1_2_data.at[FAS_2,'Space Classification'] = matching_category
               
        self.FAS_1_2_core_space = self.FAS_1_2_data.loc[self.FAS_1_2_data['Space Classification'] == 'Core']
        
        for i in self.FAS_1_2_core_space['Actual Space']:
            if not isinstance(i, float):
                print(f"Value: {i}, type: {type(i)}, building: {building_id}")
        
        # This is to handle buildings with no core space (I.e. they have no columns 
        if len(self.FAS_1_2_core_space) > 0:
            self.FAS_1_core_space = self.FAS_1_2_core_space.pivot_table(index = 'FAS 1', values = 'Actual Space', aggfunc = 'sum')
        if len(self.FAS_1_2_core_space) < 1:
            self.FAS_1_core_space = self.FAS_1_2_core_space.set_index('FAS 1')


In [43]:
# This creates a campus object for a single campus, and then calculates the reducible space and reducible combination

# CampusId can be changed via the first argument in the class:
campus = Campus('1883001', campus_properties_df, aims_report_df, FAS_2_space, FAS_1_space, building_properties_df, inclusion_exclusion_df)

print(campus.reducible_space)

print(campus.reducible_combination)

C:\Users\justi\AppData\Local\Temp\ipykernel_22380\3037506963.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FAS_2_core_entitlement["Entitled Space"] = core_entitlement_list
C:\Users\justi\AppData\Local\Temp\ipykernel_22380\3037506963.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FAS_2_core_entitlement.loc[:, "Entitled Space"] = core_entitlement_list
C:\Users\justi\AppData\Local\Temp\ipykernel_22380\3037506963.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

1576.4
('105585', '463586', '105303')


In [44]:
campus.combinations[('105585', '463586', '105303')].reducible_classification

True

In [45]:
# This returns the reducible space and classification for a given combination (True = Reducible, False = excluded), as per the campus defined in the above cell.
# Copy the desired combination below to view results

combination =  ('105585', '463586', '105303')

print(campus.combinations[combination].reducible_space, campus.combinations[combination].reducible_classification)

campus.combinations

1576.4 True


{('105585',): <__main__.Combination at 0x2dcd5c83390>,
 ('105304',): <__main__.Combination at 0x2dcd5c6c0d0>,
 ('463586',): <__main__.Combination at 0x2dcd78feb50>,
 ('105303',): <__main__.Combination at 0x2dcd5ced490>,
 ('105585', '105304'): <__main__.Combination at 0x2dcd78fcf50>,
 ('105585', '463586'): <__main__.Combination at 0x2dcd5c82e50>,
 ('105585', '105303'): <__main__.Combination at 0x2dcdfdbb490>,
 ('105304', '463586'): <__main__.Combination at 0x2dcdfdb8410>,
 ('105304', '105303'): <__main__.Combination at 0x2dcdfdb9010>,
 ('463586', '105303'): <__main__.Combination at 0x2dcdfdb9c10>,
 ('105585', '105304', '463586'): <__main__.Combination at 0x2dcd74d1150>,
 ('105585', '105304', '105303'): <__main__.Combination at 0x2dcd74d0c50>,
 ('105585', '463586', '105303'): <__main__.Combination at 0x2dcd74d2cd0>,
 ('105304', '463586', '105303'): <__main__.Combination at 0x2dcd74d3f90>,
 ('105585',
  '105304',
  '463586',
  '105303'): <__main__.Combination at 0x2dcd74d0b10>}

In [46]:
inclusion_exclusion_df.loc[inclusion_exclusion_df["column_name"] == "primary_enrolments",  "exclude_true_false"] = True

inclusion_exclusion_df.loc[inclusion_exclusion_df["column_name"] == "primary_enrolments",  "num_range_stop"] = 50

In [49]:
# This calculates the included campuses for the portfolio reducible space calc

campus_list = campus_properties_df["CampusId"].unique()

campus_exclusions = inclusion_exclusion_df.loc[(inclusion_exclusion_df["table_name"] == "Campus Properties") & (inclusion_exclusion_df["exclude_true_false"] == True)]

numeric_exclusions = campus_exclusions.loc[inclusion_exclusion_df["exclusion_type"] == 'num_range']

string_exclusions = campus_exclusions.loc[inclusion_exclusion_df["exclusion_type"] == 'string_value']

exclusion_list = [] # <-- This is the list where excluded campuses are stored

# iterates through the columns in the numeric exclusions included in the inclusion_exclusion dataframe
for exclusion in numeric_exclusions["column_name"]:

    # checks if the exclusion is included in the campus_properties dataframe
    if exclusion in list(campus_properties_df.columns):

        # defines the lower & upper thresholds for the exclusion criteria ( .iloc[0] is used to retrieve the first value from the pd.Series that is returned by .loc )
        low_value = numeric_exclusions.loc[numeric_exclusions["column_name"] == exclusion, "num_range_start"].iloc[0]
        high_value = numeric_exclusions.loc[numeric_exclusions["column_name"] == exclusion, "num_range_stop"].iloc[0]

        # Iterates through the campuses in the campus_properties dataframe; actual_value is the number for the given campus
        for campus in campus_list:
            actual_value = campus_properties_df.loc[campus_properties_df["CampusId"] == campus, exclusion].iloc[0]

            # checks if value is within exclusion range
            try:
                if actual_value >= low_value and actual_value <= high_value:
                    exclusion_list.append(campus)
            except:
                print(f"Actual value: {type(actual_value)}, high-low: {high_value},{low_value}")

# iterates through the columns in the string_value exclusions included in the inclusion_exclusion dataframe
for exclusion in string_exclusions["column_name"]:

    # checks if the exclusion is included in the campus_properties dataframe
    if exclusion in list(campus_properties_df.columns):

        # This retrieve the comma-separated strings
        comma_separated_strings = string_exclusions.loc[string_exclusions["column_name"] == exclusion, "string_value_criteria"].iloc[0]

        # This breaks up the string into the individual and strips any spaces from the start/finish
        string_list = comma_separated_strings.split(",")
        string_list = [s.strip() for s in string_list]

        for campus in campus_list:
            actual_value = campus_properties_df.loc[campus_properties_df["CampusId"] == campus, exclusion].iloc[0]

            # checks if the campus attribute (actual_value) is in the list of exclusions
            try:
                if actual_value in string_list:
                    _exclusion_list.append(campus)
            except:
                print(f"Campus: {campus}, Actual value: {actual_value}, Exclusion list: {string_list}") # <-- This prints the attributes that caused an error to occur
                
campus_exclusion_list = exclusion_list

campus_included_list = [c for c in campus_list if c not in campus_exclusion_list]

print("Campus Inclusion List:", campus_included_list, "\n")

print("Campus Exclusion List:", campus_exclusion_list, "\n")

Campus Inclusion List: ['1550501', '1883001'] 

Campus Exclusion List: ['1841501'] 



In [50]:
campus_dictionary = { "CampusId" : [],
                     "Campus Object": [],
                     "Reducible Space" : [],
                     "Reducible Buildings" : [] }


for campus in campus_included_list:
    campus_dictionary["CampusId"].append(campus)
    
    campus_object = Campus(campus, campus_properties_df, aims_report_df, FAS_2_space, FAS_1_space, building_properties_df, inclusion_exclusion_df)
    
    campus_dictionary["Campus Object"].append(campus_object)
    
    if campus_object.percentage_excess_core_space < 1:
        
        campus_dictionary["Reducible Space"].append(0)
        
        campus_dictionary["Reducible Buildings"].append(())
        
    else:
        campus_dictionary["Reducible Space"].append(campus_object.reducible_space)
        
        campus_dictionary["Reducible Buildings"].append(campus_object.reducible_combination)

C:\Users\justi\AppData\Local\Temp\ipykernel_22380\3037506963.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FAS_2_core_entitlement["Entitled Space"] = core_entitlement_list
C:\Users\justi\AppData\Local\Temp\ipykernel_22380\3037506963.py:323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FAS_2_core_entitlement.loc[:, "Entitled Space"] = core_entitlement_list
C:\Users\justi\AppData\Local\Temp\ipykernel_22380\3037506963.py:322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [52]:
campus_reducible_space_df = pd.DataFrame(campus_dictionary)

portfolio_reducible_space = campus_reducible_space_df["Reducible Space"].sum()

portfolio_reducible_space

1576.4

In [57]:
campus_reducible_space_df

# Copy the desired file path here. Make sure it includes the folder and the name of the file. Example: r"C:\Users\justi\Desktop\Reducible Space Folder\Test 123.csv"
download_file_path = r"__file_path__\__file_name__.csv "

campus_reducible_space_df.to_csv(download_file_path, index = False)